In [123]:
from pybtex.database.input import bibtex
import pybtex.database.input.bibtex 
from time import strptime
import string
import html
import os

In [138]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

In [165]:
#open a bibtex file
parser = bibtex.Parser()

#bibdata = parser.parse_file("pubs.bib")
bibdata = parser.parse_file("proceedings.bib")
(proceedings,journal) = (True, False)

pub_year = "2000"
pub_month = "01"
pub_day = "01"
import re
#loop through the individual references
for bib_id in bibdata.entries:
    b = bibdata.entries[bib_id].fields
    print("%#"*40)
    print(b)
    try:
        # change these lines to create a SQL insert
 #       print(f'{b["year"]}-{b["month"]}\t{b["title"].replace("{", "").replace("}","")}')
 #       print(b["journal"])
        pub_year = f'{b["year"]}'
        if "month" in b.keys(): 
            print(b["month"])
            if(len(b["month"])<3):
                pub_month = "0"+b["month"]
                pub_month = pub_month[-2:]
            elif(b["month"] not in range(12)):
                tmnth = strptime(b["month"][:3],'%b').tm_mon   
                pub_month = "{:02d}".format(tmnth) 
            else:
                pub_month = str(b["month"])
        if "day" in b.keys(): 
            pub_day = str(b["day"])
        clean_title = b["title"].replace("{", "").replace("}","").replace("\\","").replace(" ","-")    
        url_slug = re.sub("\\[.*\\]|[^a-zA-Z0-9_-]", "", clean_title)
        url_slug = url_slug.replace("--","-")
    
        pub_date = pub_year+"-"+pub_month+"-"+pub_day
        md_filename = (str(pub_date) + "-" + url_slug + ".md").replace("--","-")
        html_filename = (str(pub_date) + "-" + url_slug).replace("--","-")

        citation = ""
        for author in bibdata.entries[bib_id].persons["author"]:
            citation = citation+" "+author.first_names[0]+" "+author.last_names[0]+", "
        citation = citation + "\"" + html_escape(b["title"].replace("{", "").replace("}","").replace("\\","")) + ".\""
        if(proceedings):
            venue = b["booktitle"].replace("{", "").replace("}","").replace("\\","")
        elif(journal):
            venue = b["journal"]
        else:
            venue = ""
        citation = citation + " " + html_escape(venue)

        citation = citation + ", " + pub_year + "."
        
        print(citation)
        ## YAML variables

        md = "---\ntitle: \""   + html_escape(b["title"].replace("{", "").replace("}","").replace("\\","")) + '"\n'

        md += """collection: publications"""

        md += """\npermalink: /publication/""" + html_filename

        if "note" in b.keys():
            if len(str(b["note"])) > 5:
                md += "\nexcerpt: '" + html_escape(b["note"]) + "'"

        md += "\ndate: " + str(pub_date) 

        md += "\nvenue: '" + html_escape(venue) + "'"

        if "url" in b.keys():
            if len(str(b["url"])) > 5:
                md += "\npaperurl: '" + b["url"] + "'"

        md += "\ncitation: '" + html_escape(citation) + "'"

        md += "\n---"

        ## Markdown description for individual page
        if "note" in b.keys():
            if len(str(b["note"])) > 5:
                md += "\n" + html_escape(b["note"]) + "\n"

        if "url" in b.keys():
            if len(str( b["url"])) > 5:
                md += "\n[Download paper here](" + b["url"] + ")\n" 
        md += "\nUse [Google Scholar](https://scholar.google.com/scholar?q="+html.escape(clean_title.replace("-","+"))+") for full citation"

        md_filename = os.path.basename(md_filename)

        with open("../_publications/" + md_filename, 'w') as f:
            f.write(md)
        
        print(md_filename)
        print(md)
    
    # field may not exist for a reference
    except(KeyError):
        print(KeyError.args)                  
        continue
    print("%#"*40)


%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#
OrderedCaseInsensitiveDict([('Author+an', '2=highlight'), ('booktitle', '2019 International Conference on VLSI Design (VLSID 2019)'), ('title', 'A State Encoding Methodology for Side-Channel Security vs. Power Trade-off Exploration'), ('year', '2019'), ('month', '1'), ('day', '8'), ('organization', 'IEEE'), ('note', 'Awaiting proceedings \\url{https://goo.gl/4kDDec} .')])
1
 Richa Agrawal,  Mike Borowczak,  Ranga Vemuri, "A State Encoding Methodology for Side-Channel Security vs. Power Trade-off Exploration." 2019 International Conference on VLSI Design (VLSID 2019), 2019.
2019-01-8-A-State-Encoding-Methodology-for-Side-Channel-Security-vs-Power-Trade-off-Exploration.md
---
title: "A State Encoding Methodology for Side-Channel Security vs. Power Trade-off Exploration"
collection: publications
permalink: /publication/2019-01-8-A-State-Encoding-Methodology-for-Side-Channel-Security-vs-Power-Trade-off-Explora